In [401]:
import pandas as pd
import numpy as np
from operator import itemgetter
import plotly.express as px
import plotly
import plotly.graph_objs as go
from scipy import stats

In [402]:
#read data

covid_df = pd.read_csv("../../../data/output/covid.csv")
covid_df = covid_df.drop("Unnamed: 0",axis =1)
covid_df = covid_df.drop(covid_df.loc[covid_df['County Name'] == "Statewide Unallocated"].index)
covid_df.head(3)

,countyFIPS,County Name,State,population,stateFIPS,1/22/20_x,1/23/20_x,1/24/20_x,1/25/20_x,1/26/20_x,...,10/10/20_y,10/11/20_y,10/12/20_y,10/13/20_y,10/14/20_y,10/15/20_y,10/16/20_y,10/17/20_y,10/18/20_y,10/19/20_y
1,1001,Autauga County,AL,55869,1,0,0,0,0,0,...,28,28,28,28,28,28,28,28,28,28
2,1003,Baldwin County,AL,223234,1,0,0,0,0,0,...,65,65,65,65,66,66,67,67,67,67
3,1005,Barbour County,AL,24686,1,0,0,0,0,0,...,9,9,9,9,9,9,9,9,9,9


In [403]:
#Groupby covid_df based on State

covid_dfState = covid_df.groupby("State").sum()
covid_dfState = covid_dfState.drop(["countyFIPS", "stateFIPS"],axis =1)
covid_dfState.reset_index(inplace=True)
covid_dfState.head()

,State,population,1/22/20_x,1/23/20_x,1/24/20_x,1/25/20_x,1/26/20_x,1/27/20_x,1/28/20_x,1/29/20_x,...,10/10/20_y,10/11/20_y,10/12/20_y,10/13/20_y,10/14/20_y,10/15/20_y,10/16/20_y,10/17/20_y,10/18/20_y,10/19/20_y
0,AK,731545,0,0,0,0,0,0,0,0,...,59,59,59,59,62,63,65,67,67,67
1,AL,4903185,0,0,0,0,0,0,0,0,...,2664,2664,2665,2665,2706,2756,2786,2788,2788,2789
2,AR,3017804,0,0,0,0,0,0,0,0,...,1546,1561,1578,1602,1625,1637,1658,1684,1704,1714
3,AZ,7278717,0,0,0,0,1,1,1,1,...,5757,5757,5759,5767,5772,5789,5806,5822,5827,5830
4,CA,39512223,0,0,0,0,2,3,3,4,...,16578,16586,16611,16677,16772,16851,16921,16955,16991,17013


# Task1

## Generate weekly statistics (mean, median, mode) for number of cases and deaths across a specific state. Compare the data against other states.

#### I selected MA to generate weekly statistics. Then, I chose AZ, IN, MO, NJ, TN to compare their data with MA data. The reason that I selected those 5 states is that their population is similar to MA.

In [404]:
# Select states AZ, IN, MO, NJ, TN

covid_dfSt = covid_dfState.iloc[ [3,15,19,24,31,42] , : ]
covid_dfSt

,State,population,1/22/20_x,1/23/20_x,1/24/20_x,1/25/20_x,1/26/20_x,1/27/20_x,1/28/20_x,1/29/20_x,...,10/10/20_y,10/11/20_y,10/12/20_y,10/13/20_y,10/14/20_y,10/15/20_y,10/16/20_y,10/17/20_y,10/18/20_y,10/19/20_y
3,AZ,7278717,0,0,0,0,1,1,1,1,...,5757,5757,5759,5767,5772,5789,5806,5822,5827,5830
15,IN,6732219,0,0,0,0,0,0,0,0,...,3781,3788,3794,3821,3835,3862,3887,3918,3937,3960
19,MA,6892503,0,0,0,0,0,0,0,0,...,9582,9599,9612,9625,9642,9667,9697,9718,9732,9748
24,MO,6137428,0,0,0,0,0,0,0,0,...,2413,2413,2413,2413,2544,2544,2544,2549,2549,2549
31,NJ,8882190,0,0,0,0,0,0,0,0,...,16167,16170,16170,16177,16191,16197,16202,16204,16210,16214
42,TN,6829174,0,0,0,0,0,0,0,0,...,2719,2728,2734,2757,2788,2825,2836,2866,2872,2884


In [405]:
#Seperate Cases and Deaths

StCases = covid_dfSt.filter(regex = "x")
StDeaths = covid_dfSt.filter(regex = "y")

In [406]:
# Calculate new Cases and deaths

StNewCases = StCases.diff(axis = 1)
StNewDeaths = StDeaths.diff(axis = 1)
StNewCases

,1/22/20_x,1/23/20_x,1/24/20_x,1/25/20_x,1/26/20_x,1/27/20_x,1/28/20_x,1/29/20_x,1/30/20_x,1/31/20_x,...,10/10/20_x,10/11/20_x,10/12/20_x,10/13/20_x,10/14/20_x,10/15/20_x,10/16/20_x,10/17/20_x,10/18/20_x,10/19/20_x
3,NaN,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,886.0,605.0,475.0,681.0,904.0,1075.0,724.0,972.0,741.0,716.0
15,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1925.0,1570.0,1574.0,1572.0,1165.0,1947.0,2298.0,2485.0,1605.0,1583.0
19,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,592.0,558.0,778.0,619.0,2525.0,571.0,687.0,567.0,744.0,827.0
24,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4988.0,0.0,0.0,0.0,4568.0,1875.0,0.0,4396.0,1769.0,0.0
31,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,873.0,742.0,471.0,967.0,949.0,969.0,815.0,946.0,1249.0,1189.0
42,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1708.0,1858.0,2660.0,1190.0,1791.0,2162.0,703.0,2528.0,2405.0,2951.0


#### Normalizing new cases and new deaths per 1000000 people. Since population of each state is around 7000000, I selected 1000000 for normalizing my data.

In [407]:
# Normalize new cases

StNormCases = StNewCases.transpose()
StNormCases["AZ Cases(Norm)"] = ((StNormCases[3]/7278717)*1000000).round()
StNormCases["IN Cases(Norm)"] = ((StNormCases[15]/6732219)*1000000).round()
StNormCases["MA Cases(Norm)"] = ((StNormCases[19]/6892503)*1000000).round()
StNormCases["MO Cases(Norm)"] = ((StNormCases[24]/6137428)*1000000).round()
StNormCases["NJ Cases(Norm)"] = ((StNormCases[31]/8882190)*1000000).round()
StNormCases["TN Cases(Norm)"] = ((StNormCases[42]/6829174)*1000000).round()
StNormCases.drop(columns =[3,15,19,24,31,42], inplace = True)


# Normalize new deaths

StNormDeaths = StNewDeaths.transpose()
StNormDeaths["AZ Deaths(Norm)"] = ((StNormDeaths[3]/7278717)*1000000).round()
StNormDeaths["IN Deaths(Norm)"] = ((StNormDeaths[15]/6732219)*1000000).round()
StNormDeaths["MA Deaths(Norm)"] = ((StNormDeaths[19]/6892503)*1000000).round()
StNormDeaths["MO Deaths(Norm)"] = ((StNormDeaths[24]/6137428)*1000000).round()
StNormDeaths["NJ Deaths(Norm)"] = ((StNormDeaths[31]/8882190)*1000000).round()
StNormDeaths["TN Deaths(Norm)"] = ((StNormDeaths[42]/6829174)*1000000).round()
StNormDeaths.drop(columns =[3,15,19,24,31,42], inplace = True)

#### Calculate weekly mean for new cases and new deaths

In [408]:
#Calculate mean for Normalized New Cases

StNormCases_week = StNormCases.groupby(np.arange(len(StNormCases))//7).mean().round(0).astype(int).rename_axis('Week')
StNormCases_week.columns = ['AZ', 'IN','MA','MO','NJ','TN']
StNormCases_week = StNormCases_week.reset_index()
StNormCases_week

,Week,AZ,IN,MA,MO,NJ,TN
0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0
2,2,0,0,0,0,0,0
3,3,0,0,0,0,0,0
4,4,0,0,0,0,0,0
5,5,0,0,0,0,0,0
6,6,0,0,2,0,0,0
7,7,0,0,3,0,4,1
8,8,6,7,18,5,45,10
9,9,19,38,108,25,193,25


In [441]:
#Calculate mean for Normalized New Deaths

StNormDeaths_week = StNormDeaths.groupby(np.arange(len(StNormDeaths))//7).mean().round(0).astype(int).rename_axis('Week')
StNormDeaths_week.columns = ['AZ', 'IN','MA','MO','NJ','TN']
StNormDeaths_week = StNormDeaths_week.reset_index()
StNormDeaths_week

,Week,AZ,IN,MA,MO,NJ,TN
0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0
2,2,0,0,0,0,0,0
3,3,0,0,0,0,0,0
4,4,0,0,0,0,0,0
5,5,0,0,0,0,0,0
6,6,0,0,0,0,0,0
7,7,0,0,0,0,0,0
8,8,0,0,0,0,1,0
9,9,0,1,2,0,3,0


#### Mean, Median and Mode for normalized new cases and new deaths. Value of mode for most of the states is zero. That's why, after calculating mode, I also write a code for ignoring zero and calculating mode.

In [410]:
# Calculate Mean, Median, Mode for Normalized Cases

AZNormCases_mean = round(StNormCases_week["AZ"].mean())
AZNormCases_median = round(StNormCases_week["AZ"].median())
AZNormCases_mode = StNormCases_week["AZ"].mode()
AZNormCases0_mode = StNormCases_week["AZ"].replace(0, np.nan).mode()

INNormCases_mean = round(StNormCases_week["IN"].mean())
INNormCases_median = round(StNormCases_week["IN"].median())
INNormCases_mode = StNormCases_week["IN"].mode()
INNormCases0_mode = StNormCases_week["IN"].replace(0, np.nan).mode()

MANormCases_mean = round(StNormCases_week["MA"].mean())
MANormCases_median = round(StNormCases_week["MA"].median())
MANormCases_mode = StNormCases_week["MA"].mode()
MANormCases0_mode = StNormCases_week["MA"].replace(0, np.nan).mode()

MONormCases_mean = round(StNormCases_week["MO"].mean())
MONormCases_median = round(StNormCases_week["MO"].median())
MONormCases_mode = StNormCases_week["MO"].mode()
MONormCases0_mode = StNormCases_week["MO"].replace(0, np.nan).mode()

NJNormCases_mean = round(StNormCases_week["NJ"].mean())
NJNormCases_median = round(StNormCases_week["NJ"].median())
NJNormCases_mode = StNormCases_week["NJ"].mode()
NJNormCases0_mode = StNormCases_week["NJ"].replace(0, np.nan).mode()

TNNormCases_mean = round(StNormCases_week["TN"].mean())
TNNormCases_median = round(StNormCases_week["TN"].median())
TNNormCases_mode = StNormCases_week["TN"].mode()
TNNormCases0_mode = StNormCases_week["TN"].replace(0, np.nan).mode()

In [411]:
# Create a list of lists for cases mean, median, mode

Cases_statistics =[[AZNormCases_mean, INNormCases_mean, MANormCases_mean, MONormCases_mean, NJNormCases_mean,TNNormCases_mean],
             [AZNormCases_median, INNormCases_median, MANormCases_median, MONormCases_median, NJNormCases_median, TNNormCases_median],
             [AZNormCases_mode[0], INNormCases_mode[0], MANormCases_mode[0], MONormCases_mode[0], NJNormCases_mode[0], TNNormCases_mode[0]]]


#Create a dataframe from list

Cases_sta = pd.DataFrame(Cases_statistics, columns = ['AZ', 'IN', 'MA','MO', 'NJ', 'TN'], 
                         index=['Mean', 'Median', 'Mode'])  
Cases_sta

,AZ,IN,MA,MO,NJ,TN
Mean,117,84,81,95,92,122
Median,62,76,49,38,45,72
Mode,0,0,0,0,0,0


##### In all states mean is larger than median and mode is zero which means the distribution is right skewed.  Larger value of mean shows that there are a few larger value of new cases in the data which drives the mean upward, but they could not effect the median. The difference between mean and median in IN is not very big which shows that the distribution of new cases data can be assumed to be approximately symmetrical compare to other states. TN has the highest mean and MA lowest mean among other states. Since the data is normalized, we can conclude that MA has the lowest new cases and TN has the highest new Cases among other states.

In [412]:
# Calculate Mean, Median, Mode for normalized new Deaths

AZNormDeaths_mean = round(StNormDeaths_week["AZ"].mean())
AZNormDeaths_median = round(StNormDeaths_week["AZ"].median())
AZNormDeaths_mode = StNormDeaths_week["AZ"].mode()
AZNormDeaths0_mode = StNormDeaths_week["AZ"].replace(0, np.nan).mode()

INNormDeaths_mean = round(StNormDeaths_week["IN"].mean())
INNormDeaths_median = round(StNormDeaths_week["IN"].median())
INNormDeaths_mode = StNormDeaths_week["IN"].mode()
INNormDeaths0_mode = StNormDeaths_week["IN"].replace(0, np.nan).mode()

MANormDeaths_mean = round(StNormDeaths_week["MA"].mean())
MANormDeaths_median = round(StNormDeaths_week["MA"].median())
MANormDeaths_mode = StNormDeaths_week["MA"].mode()
MANormDeaths0_mode = StNormDeaths_week["MA"].replace(0, np.nan).mode()

MONormDeaths_mean = round(StNormDeaths_week["MO"].mean())
MONormDeaths_median = round(StNormDeaths_week["MO"].median())
MONormDeaths_mode = StNormDeaths_week["MO"].mode()
MONormDeaths0_mode = StNormDeaths_week["MO"].replace(0, np.nan).mode()

NJNormDeaths_mean = round(StNormDeaths_week["NJ"].mean())
NJNormDeaths_median = round(StNormDeaths_week["NJ"].median())
NJNormDeaths_mode = StNormDeaths_week["NJ"].mode()
NJNormDeaths0_mode = StNormDeaths_week["NJ"].replace(0, np.nan).mode()


TNNormDeaths_mean = round(StNormDeaths_week["TN"].mean())
TNNormDeaths_median = round(StNormDeaths_week["TN"].median())
TNNormDeaths_mode = StNormDeaths_week["TN"].mode()
TNNormDeaths0_mode = StNormDeaths_week["TN"].replace(0, np.nan).mode()

# TNNormDeaths_mean = round(StNormDeaths_week["TN"].mean())
# TNNormDeaths_median = round(StNormDeaths_week["TN"].median())
# TNNormDeaths_mode = StNormDeaths_week["TN"].mode()
# TNNormDeaths0_mode = StNormDeaths_week["TN"].replace(0, np.nan).mode()

In [413]:

Cases_statistics =[[AZNormDeaths_mean, INNormDeaths_mean, MANormDeaths_mean, MONormDeaths_mean, NJNormDeaths_mean, TNNormDeaths_mean],
             [AZNormDeaths_median,INNormDeaths_median, MANormDeaths_median, MONormDeaths_median, NJNormDeaths_median, TNNormDeaths_median],
             [AZNormDeaths_mode[0],INNormDeaths_mode[0],MANormDeaths_mode[0],MONormDeaths_mode[0],NJNormDeaths_mode[0],TNNormDeaths_mode[0]]]


#Create a dataframe from list

Cases_sta = pd.DataFrame(Cases_statistics, columns = ['AZ', 'IN', 'MA','MO', 'NJ', 'TN'], 
                         index=['Mean', 'Median', 'Mode'])  
Cases_sta

,AZ,IN,MA,MO,NJ,TN
Mean,3,2,5,2,7,2
Median,2,2,2,1,1,1
Mode,0,2,2,1,0,1


##### NJ has the highest mean of new deaths and the difference between its median and its mean is remarkable compare to other states which shows the peak is close to the left side of the graph. IN, TN and MO have the lowest Mean but the mean, median and mode of IN are equal which demonestrate that  the distribution of new cases in IN as approximately symmetrical. For some period of time mode = 0 indicates that the state was successful to minimize the number of deathes for a greater number of weeks, however it was temporery.

## Identify counties within the previous state with high case and death rates.

In [433]:
#select counties for MA

MACounties = covid_df.loc[covid_df['State'] == "MA"]
MACounties.head()

,countyFIPS,County Name,State,population,stateFIPS,1/22/20_x,1/23/20_x,1/24/20_x,1/25/20_x,1/26/20_x,...,10/10/20_y,10/11/20_y,10/12/20_y,10/13/20_y,10/14/20_y,10/15/20_y,10/16/20_y,10/17/20_y,10/18/20_y,10/19/20_y
1240,25001,Barnstable County,MA,212990,25,0,0,0,0,0,...,176,176,177,178,178,178,180,181,181,181
1241,25003,Berkshire County,MA,124944,25,0,0,0,0,0,...,51,51,51,51,51,51,51,51,51,51
1242,25005,Bristol County,MA,565217,25,0,0,0,0,0,...,718,721,724,725,725,727,729,731,731,732
1243,25007,Dukes County,MA,17332,25,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1244,25009,Essex County,MA,789034,25,0,0,0,0,0,...,1307,1309,1311,1313,1315,1318,1320,1322,1326,1330


In [415]:
#add a new column to MA dataset which shows total confirmed cases for each county
MACounties['Total Cases']= MACounties["10/19/20_x"]

#add a new column to MA dataset which shows total death for each county
MACounties['Total Death']= MACounties["10/19/20_y"]
MACounties.head()

C:\Users\sanam\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\sanam\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,countyFIPS,County Name,State,population,stateFIPS,1/22/20_x,1/23/20_x,1/24/20_x,1/25/20_x,1/26/20_x,...,10/12/20_y,10/13/20_y,10/14/20_y,10/15/20_y,10/16/20_y,10/17/20_y,10/18/20_y,10/19/20_y,Total Cases,Total Death
1240,25001,Barnstable County,MA,212990,25,0,0,0,0,0,...,177,178,178,178,180,181,181,181,2123,181
1241,25003,Berkshire County,MA,124944,25,0,0,0,0,0,...,51,51,51,51,51,51,51,51,783,51
1242,25005,Bristol County,MA,565217,25,0,0,0,0,0,...,724,725,725,727,729,731,731,732,11639,732
1243,25007,Dukes County,MA,17332,25,0,0,0,0,0,...,0,0,0,0,0,0,0,0,87,0
1244,25009,Essex County,MA,789034,25,0,0,0,0,0,...,1311,1313,1315,1318,1320,1322,1326,1330,22899,1330


In [416]:
# Highest rate of cases in MA counties

highest_rateCases_MA = MACounties[['County Name', "population", "Total Cases"]] 
highest_rateCases_MA["Rate of Cases"] = ((highest_rateCases_MA["Total Cases"]/highest_rateCases_MA["population"])*100000).round()
highest_rateCases_MA = highest_rateCases_MA.sort_values(['Rate of Cases'], ascending = False) 
highest_rateCases_MA.head()

C:\Users\sanam\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,County Name,population,Total Cases,Rate of Cases
1252,Suffolk County,803907,27634,3437.0
1244,Essex County,789034,22899,2902.0
1253,Worcester County,830622,18105,2180.0
1251,Plymouth County,521202,11194,2148.0
1242,Bristol County,565217,11639,2059.0


In [417]:
# Highest rate of death in MA counties without normalizing

highest_rateDeath_MA = MACounties[['County Name', "population", "Total Death"]] 
highest_rateDeath_MA["Rate of Death"] = ((highest_rateDeath_MA["Total Death"]/highest_rateDeath_MA["population"])*100000).round()
highest_rateDeath_MA = highest_rateDeath_MA.sort_values(['Rate of Death'], ascending = False) 
highest_rateDeath_MA.head()

C:\Users\sanam\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,County Name,population,Total Death,Rate of Death
1246,Hampden County,466372,799,171.0
1244,Essex County,789034,1330,169.0
1250,Norfolk County,706775,1093,155.0
1251,Plymouth County,521202,804,154.0
1252,Suffolk County,803907,1160,144.0


## Plot daily trends (cases and deaths, new cases) of state and top 5 infected counties. Utilize aggregrate, normalized by population, and log normalized values.

### New Cases

In [418]:
stateMAST= covid_dfSt.transpose()
MA = stateMAST[19]
MA_NewCases = MA.filter(regex = "x").to_frame()
MA_NewDeaths = MA.filter(regex = "y").to_frame()

In [419]:
#Select counties with highest rate of cases 
MACounties_caseRate = MACounties.loc[(MACounties['County Name']=="Suffolk County") | (MACounties['County Name']=="Essex County")
                             | (MACounties['County Name']=="Plymouth County")| (MACounties['County Name']=="Bristol County")
                             | (MACounties['County Name']=="Middlesex County")]
MACounties_caseRate.drop(columns = "County Name_y", inplace=True)

In [420]:
#Select number of Cases
CountiesCases = MACounties_caseRate.filter(regex = "x")
CountiesCases = CountiesCases.transpose()

#Rename columns
CountiesCases.columns = ['Bristol County', 'Essex County','Middlesex County','Plymouth County'
                         ,'Suffolk County']
CountiesCases.head()

,Bristol County,Essex County,Middlesex County,Plymouth County,Suffolk County
1/22/20_x,0,0,0,0,0
1/23/20_x,0,0,0,0,0
1/24/20_x,0,0,0,0,0
1/25/20_x,0,0,0,0,0
1/26/20_x,0,0,0,0,0


In [421]:
# Calculate new Cases for highest rated counties

CountiesNewCases = CountiesCases.diff()
MA_NewCases = MA_NewCases.diff()

In [422]:
#Concatenate two MA dataframe with counties dataframe

dailyTrendsCases = pd.concat([MA_NewCases, CountiesNewCases], axis=1)
dailyTrendsCases.head()

,19,Bristol County,Essex County,Middlesex County,Plymouth County,Suffolk County
1/22/20_x,NaN,NaN,NaN,NaN,NaN,NaN
1/23/20_x,0,0.0,0.0,0.0,0.0,0.0
1/24/20_x,0,0.0,0.0,0.0,0.0,0.0
1/25/20_x,0,0.0,0.0,0.0,0.0,0.0
1/26/20_x,0,0.0,0.0,0.0,0.0,0.0


#### Normalizing Data

In [423]:
# Normalize new cases

dailyTrendsCases["MA Cases(Norm)"] = (((dailyTrendsCases[19]/6892503)*300000).astype(float)).round()
dailyTrendsCases["Bristol Cases(Norm)"] = ((dailyTrendsCases["Bristol County"]/565217)*300000).round()
dailyTrendsCases["Essex Cases(Norm)"] = ((dailyTrendsCases["Essex County"]/789034)*300000).round()
dailyTrendsCases["Middlesex Cases(Norm)"] = ((dailyTrendsCases["Middlesex County"]/1611699)*300000).round()
dailyTrendsCases["Plymouth Cases(Norm)"] = ((dailyTrendsCases["Plymouth County"]/521202)*300000).round()
dailyTrendsCases["Suffolk Cases(Norm)"] = ((dailyTrendsCases["Suffolk County"]/803907)*300000).round()


#Drop unnecessary columns 
dailyTrendsCases.drop(columns =[19,"Bristol County", "Essex County", "Middlesex County", "Plymouth County",
                               "Suffolk County"], inplace = True)

#### Plot Daily Trends ( New Cases)

In [424]:
#Plotting Daily trends 

trace0 = go.Scatter(x=dailyTrendsCases.index, y=dailyTrendsCases["MA Cases(Norm)"], 
                    name="MA", 
                    mode="lines+markers") 

trace1 = go.Scatter(x=dailyTrendsCases.index, y=dailyTrendsCases["Bristol Cases(Norm)"], 
                    name="Bristol", 
                    mode="lines+markers")  

trace2 = go.Scatter(x=dailyTrendsCases.index, y=dailyTrendsCases["Essex Cases(Norm)"], 
                    name="Essex", 
                    mode="lines+markers") 

trace3 = go.Scatter(x=dailyTrendsCases.index, y=dailyTrendsCases["Middlesex Cases(Norm)"], 
                    name="Middlesex", 
                    mode="lines+markers") 

trace4 = go.Scatter(x=dailyTrendsCases.index, y=dailyTrendsCases["Plymouth Cases(Norm)"], 
                    name="Plymouth", 
                    mode="lines+markers") 

trace5 = go.Scatter(x=dailyTrendsCases.index, y=dailyTrendsCases["Suffolk Cases(Norm)"], 
                    name="Suffolk", 
                    mode="lines+markers") 


mydata = go.Data([trace0, trace1,trace2, trace3,trace4,trace5])

mylayout = go.Layout(
    title=""
)

fig = go.Figure(data=mydata, layout=mylayout)

plotly.offline.iplot(fig, filename = '')

#### New Deaths

In [425]:
#Select counties with highest rate of cases 
MACounties_deathsRate = MACounties.loc[(MACounties['County Name']=="Essex County") | (MACounties['County Name']=="Hampden County")
                             | (MACounties['County Name']=="Norfolk County")| (MACounties['County Name']=="Plymouth County")
                             | (MACounties['County Name']=="Suffolk County")]
MACounties_deathsRate.drop(columns = "County Name_y", inplace=True)

In [435]:
#Select number of Deaths
CountiesDeaths = MACounties_deathsRate.filter(regex = "_y")
CountiesDeaths = CountiesDeaths.transpose()

#Rename columns
CountiesDeaths.columns = ['Essex County', 'Hampden County','Norfolk County','Plymouth County'
                         ,'Suffolk County']

In [427]:
# Calculate new Deaths for highest rated counties

CountiesNewDeaths = CountiesDeaths.diff()
MA_NewDeaths = MA_NewDeaths.diff()

In [428]:
#Concatenate two MA dataframe with counties dataframe

dailyTrendsDeaths = pd.concat([MA_NewDeaths, CountiesNewDeaths], axis=1)
dailyTrendsDeaths.head()

,19,Essex County,Hampden County,Norfolk County,Plymouth County,Suffolk County
1/22/20_y,NaN,NaN,NaN,NaN,NaN,NaN
1/23/20_y,0,0.0,0.0,0.0,0.0,0.0
1/24/20_y,0,0.0,0.0,0.0,0.0,0.0
1/25/20_y,0,0.0,0.0,0.0,0.0,0.0
1/26/20_y,0,0.0,0.0,0.0,0.0,0.0


#### Normalizing Data

In [429]:
# Normalize new deaths

dailyTrendsDeaths["MA Deaths(Norm)"] = (((dailyTrendsDeaths[19]/6892503)*300000).astype(float)).round()
dailyTrendsDeaths["Essex Cases(Norm)"] = ((dailyTrendsDeaths["Essex County"]/789034)*300000).round()
dailyTrendsDeaths["Hampden Cases(Norm)"] = ((dailyTrendsDeaths["Hampden County"]/466372)*300000).round()
dailyTrendsDeaths["Norfolk Cases(Norm)"] = ((dailyTrendsDeaths["Norfolk County"]/706775)*300000).round()
dailyTrendsDeaths["Plymouth Cases(Norm)"] = ((dailyTrendsDeaths["Plymouth County"]/521202)*300000).round()
dailyTrendsDeaths["Suffolk Cases(Norm)"] = ((dailyTrendsDeaths["Suffolk County"]/803907)*300000).round()

#Drop unnecessary columns 
dailyTrendsDeaths.drop(columns =[19, "Essex County", "Hampden County", "Norfolk County", "Plymouth County",
                               "Suffolk County"], inplace = True)
dailyTrendsDeaths.head()

,MA Deaths(Norm),Essex Cases(Norm),Hampden Cases(Norm),Norfolk Cases(Norm),Plymouth Cases(Norm),Suffolk Cases(Norm)
1/22/20_y,NaN,NaN,NaN,NaN,NaN,NaN
1/23/20_y,0.0,0.0,0.0,0.0,0.0,0.0
1/24/20_y,0.0,0.0,0.0,0.0,0.0,0.0
1/25/20_y,0.0,0.0,0.0,0.0,0.0,0.0
1/26/20_y,0.0,0.0,0.0,0.0,0.0,0.0


#### Plot Daily Trends (New Deaths)

In [430]:
#Plotting Daily trends 

trace0 = go.Scatter(x=dailyTrendsDeaths.index, y=dailyTrendsDeaths["MA Deaths(Norm)"], 
                    name="MA", 
                    mode="lines+markers") 

trace1 = go.Scatter(x=dailyTrendsDeaths.index, y=dailyTrendsDeaths["Essex Cases(Norm)"], 
                    name="Essex", 
                    mode="lines+markers")  

trace2 = go.Scatter(x=dailyTrendsDeaths.index, y=dailyTrendsDeaths["Hampden Cases(Norm)"], 
                    name="Hampden", 
                    mode="lines+markers") 

trace3 = go.Scatter(x=dailyTrendsDeaths.index, y=dailyTrendsDeaths["Norfolk Cases(Norm)"], 
                    name="Norfolk", 
                    mode="lines+markers") 

trace4 = go.Scatter(x=dailyTrendsDeaths.index, y=dailyTrendsDeaths["Plymouth Cases(Norm)"], 
                    name="Plymouth", 
                    mode="lines+markers") 

trace5 = go.Scatter(x=dailyTrendsDeaths.index, y=dailyTrendsDeaths["Suffolk Cases(Norm)"], 
                    name="Suffolk", 
                    mode="lines+markers") 


mydata = go.Data([trace0, trace1,trace2, trace3,trace4,trace5])

mylayout = go.Layout(
    title=""
)

fig = go.Figure(data=mydata, layout=mylayout)

plotly.offline.iplot(fig, filename = '')

# Task 2

## Fit a distribution to the number of COVID-19 cases of a state.

Graphically plot the distribution and describe the distribution statistics. If using discrete values, calculate the Probability Mass Function for the individual values or range (if using histogram) and plot that. What do we need to plot here?


## Model a poission distribution of COVID-19 cases and deaths of a state and compare to other 5 states.

#### Normalizing Data by per 1000000 people

In [449]:
# StNormCases_week
# StNormDeaths_week

In [450]:
#Calculate mean of cases for each state 

STCasesMean = StNormCases_week.mean().round().astype(int)
STCasesMean

Week     19
AZ      117
IN       84
MA       81
MO       95
NJ       92
TN      122
dtype: int32

In [451]:
#Calculate mean of deaths for each state 

STDeathsMean = StNormDeaths_week.mean().round()
STDeathsMean

Week    19.0
AZ       3.0
IN       2.0
MA       5.0
MO       2.0
NJ       7.0
TN       2.0
dtype: float64

In [455]:
#calculate different K for different states for cases

K = [50,60,70,80,90,100,120,130]
AZ = []
IN = []
MA = []
MO = []
NJ = []
TN = []

# append calculated k to each corresponding list
for num in K:
    PMF = stats.poisson.pmf(k=num, mu=STCasesMean["AZ"]).round(4)
    AZ.append(PMF)
    
for num in K:
    PMF = stats.poisson.pmf(k=num, mu=STCasesMean["IN"]).round(4)
    IN.append(PMF)

for num in K:
    PMF = stats.poisson.pmf(k=num, mu=STCasesMean["MA"]).round(4)
    MA.append(PMF)
    
for num in K:
    PMF = stats.poisson.pmf(k=num, mu=STCasesMean["MO"]).round(4)
    MO.append(PMF)

for num in K:
    PMF = stats.poisson.pmf(k=num, mu=STCasesMean["NJ"]).round(4)
    NJ.append(PMF)
    
for num in K:
    PMF = stats.poisson.pmf(k=num, mu=STCasesMean["TN"]).round(4)
    TN.append(PMF)
    
#create a dataframe from lists
dfCases = pd.DataFrame(list(zip(K, AZ, IN, MA, MO, NJ, TN)), columns =['k','AZ', 'IN', 'MA', 'MO', 'NJ', 'TN'])
dfCases

,k,AZ,IN,MA,MO,NJ,TN
0,50,0.0000,0.0000,0.0001,0.0000,0.0000,0.0000
1,60,0.0000,0.0011,0.0026,0.0000,0.0001,0.0000
2,70,0.0000,0.0138,0.0218,0.0013,0.0027,0.0000
3,80,0.0001,0.0404,0.0443,0.0127,0.0196,0.0000
4,90,0.0014,0.0341,0.0259,0.0367,0.0411,0.0004
5,100,0.0109,0.0095,0.0050,0.0350,0.0284,0.0048
6,120,0.0350,0.0000,0.0000,0.0018,0.0007,0.0358
7,130,0.0174,0.0000,0.0000,0.0001,0.0000,0.0270


In [458]:
#calculate different K for different states for cases

KDeaths = [0,1,2,3,4,5,6,7]
AZdeaths = []
INdeaths = []
MAdeaths = []
MOdeaths = []
NJdeaths = []
TNdeaths = []

# append calculated k to each corresponding list
for num in KDeaths:
    PMF = stats.poisson.pmf(k=num, mu=STDeathsMean["AZ"]).round(4)
    AZdeaths.append(PMF)
    
for num in KDeaths:
    PMF = stats.poisson.pmf(k=num, mu=STDeathsMean["IN"]).round(4)
    INdeaths.append(PMF)

for num in KDeaths:
    PMF = stats.poisson.pmf(k=num, mu=STDeathsMean["MA"]).round(4)
    MAdeaths.append(PMF)
    
for num in KDeaths:
    PMF = stats.poisson.pmf(k=num, mu=STDeathsMean["MO"]).round(4)
    MOdeaths.append(PMF)

for num in KDeaths:
    PMF = stats.poisson.pmf(k=num, mu=STDeathsMean["NJ"]).round(4)
    NJdeaths.append(PMF)
    
for num in KDeaths:
    PMF = stats.poisson.pmf(k=num, mu=STDeathsMean["TN"]).round(4)
    TNdeaths.append(PMF)
    
#create a dataframe from lists
dfDeaths = pd.DataFrame(list(zip(KDeaths, AZdeaths, INdeaths, MAdeaths, MOdeaths, NJdeaths, TNdeaths)),
                  columns =['k','AZ', 'IN', 'MA', 'MO', 'NJ', 'TN'])
dfDeaths

,k,AZ,IN,MA,MO,NJ,TN
0,0,0.0498,0.1353,0.0067,0.1353,0.0009,0.1353
1,1,0.1494,0.2707,0.0337,0.2707,0.0064,0.2707
2,2,0.2240,0.2707,0.0842,0.2707,0.0223,0.2707
3,3,0.2240,0.1804,0.1404,0.1804,0.0521,0.1804
4,4,0.1680,0.0902,0.1755,0.0902,0.0912,0.0902
5,5,0.1008,0.0361,0.1755,0.0361,0.1277,0.0361
6,6,0.0504,0.0120,0.1462,0.0120,0.1490,0.0120
7,7,0.0216,0.0034,0.1044,0.0034,0.1490,0.0034


## Model poission distributions for North Carolina counties COVID-19 in cases and deaths.

In [480]:
#Select rows which are for NC

covid_dfNC = covid_df.loc[covid_df['State'] == "NC"]
covid_dfNC = covid_dfNC.drop(columns = "County Name_y")
covid_dfNC.head()

,countyFIPS,County Name,State,population,stateFIPS,1/22/20_x,1/23/20_x,1/24/20_x,1/25/20_x,1/26/20_x,...,10/10/20_y,10/11/20_y,10/12/20_y,10/13/20_y,10/14/20_y,10/15/20_y,10/16/20_y,10/17/20_y,10/18/20_y,10/19/20_y
1926,37001,Alamance County,NC,169509,37,0,0,0,0,0,...,59,59,59,59,59,59,59,59,66,68
1927,37003,Alexander County,NC,37497,37,0,0,0,0,0,...,8,8,8,8,8,9,9,9,9,9
1928,37005,Alleghany County,NC,11137,37,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
1929,37007,Anson County,NC,24446,37,0,0,0,0,0,...,6,6,6,8,8,8,8,8,8,8
1930,37009,Ashe County,NC,27203,37,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1


In [481]:
#Seperate covid_dfNC to cases number and deaths number

NCcases = covid_dfNC.filter(regex = "x")
NCdeaths = covid_dfNC.filter(regex = "_y")

NCcases.head(2)

,1/22/20_x,1/23/20_x,1/24/20_x,1/25/20_x,1/26/20_x,1/27/20_x,1/28/20_x,1/29/20_x,1/30/20_x,1/31/20_x,...,10/10/20_x,10/11/20_x,10/12/20_x,10/13/20_x,10/14/20_x,10/15/20_x,10/16/20_x,10/17/20_x,10/18/20_x,10/19/20_x
1926,0,0,0,0,0,0,0,0,0,0,...,4377,4426,4464,4512,4555,4580,4675,4757,4790,4815
1927,0,0,0,0,0,0,0,0,0,0,...,585,589,593,608,623,636,653,671,698,706


In [482]:
#Calculae new cases and new deaths

NCNewCases = NCcases.diff(axis = 1)
NCNewDeaths = NCdeaths.diff(axis = 1)

In [486]:
# Normalize new cases

NCNormCases = NCNewCases.transpose()
NCCounties = covid_dfNC["County Name"].tolist()
NCNormCases.columns = NCCounties
CountiesPopulation = covid_dfNC["population"].tolist()
NCNormCases = NCNormCases.rename(columns={col: col.split(' ')[0] for col in NCNormCases.columns})
counties = NCNormCases.columns.tolist()
ExColumns = (NCNormCases.columns).tolist()

for i in range(len(NCNormCases.columns)):
    NCNormCases[NCNormCases.columns[i]+" Cases(Norm)"]=((NCNormCases[NCNormCases.columns[i]]/CountiesPopulation[i])*30000).round()

NCNormCases.drop(columns =ExColumns, inplace = True)


# Normalize new cases

NCNormDeaths = NCNewDeaths.transpose()
# # NCCounties = covid_dfNC["County Name"].tolist()
NCNormDeaths.columns = NCCounties
# # CountiesPopulation = covid_dfNC["population"].tolist()
NCNormDeaths = NCNormDeaths.rename(columns={col: col.split(' ')[0] for col in NCNormDeaths.columns})
ExColumns = (NCNormDeaths.columns).tolist()

for i in range(len(NCNormDeaths.columns)):
    NCNormDeaths[NCNormDeaths.columns[i]+" Deaths(Norm)"]=((NCNormDeaths[NCNormDeaths.columns[i]]/CountiesPopulation[i])*30000).round()

NCNormDeaths.drop(columns =ExColumns, inplace = True)

In [487]:
#Calculate weekly mean for Normalized New Cases

NCNormCases_week = NCNormCases.groupby(np.arange(len(NCNormCases))//7).mean().round(0).astype(int).rename_axis('Week')

NCNormCases_week = NCNormCases_week.reset_index()

In [489]:
#Calculate weekly mean for Normalized New Deaths

NCNormDeaths_week = NCNormDeaths.groupby(np.arange(len(NCNormDeaths))//7).mean().round(0).astype(int).rename_axis('Week')

NCNormDeaths_week = NCNormDeaths_week.reset_index()

,Week,Alamance Deaths(Norm),Alexander Deaths(Norm),Alleghany Deaths(Norm),Anson Deaths(Norm),Ashe Deaths(Norm),Avery Deaths(Norm),Beaufort Deaths(Norm),Bertie Deaths(Norm),Bladen Deaths(Norm),...,Vance Deaths(Norm),Wake Deaths(Norm),Warren Deaths(Norm),Washington Deaths(Norm),Watauga Deaths(Norm),Wayne Deaths(Norm),Wilkes Deaths(Norm),Wilson Deaths(Norm),Yadkin Deaths(Norm),Yancey Deaths(Norm)
34,34,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
35,35,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
36,36,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
37,37,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
38,38,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [500]:
# Calculate mean of cases for each county
NCNormCases_mean = NCNormCases_week.mean().round()

#Calculate mean of deaths for each county 
NCNormDeaths_mean = NCNormDeaths_week.mean().round()

In [501]:
#model poisoon for different K for different states for cases

K = [0,1,2,3,4]
NC = []


# append calculated k to each corresponding list
for county in counties:
    countyK = []
    for num in K:
        PMF = stats.poisson.pmf(k=num, mu=NCNormCases_mean[county +" Cases(Norm)"]).round(4)
        countyK.append(PMF)
    NC.append(countyK)
          
# NC

# #create a dataframe from lists

NCCases = pd.DataFrame(NC, columns = ['K=0', 'K=1', 'K=2','K=3','K=4']) 
NCCases["County Name"] = counties
NCCases.set_index("County Name", inplace = True)
NCCases.head()

,K=0,K=1,K=2,K=3,K=4
County Name,,,,,
Alamance,0.0498,0.1494,0.2240,0.2240,0.1680
Alexander,0.1353,0.2707,0.2707,0.1804,0.0902
Alleghany,0.0498,0.1494,0.2240,0.2240,0.1680
Anson,0.0498,0.1494,0.2240,0.2240,0.1680
Ashe,0.3679,0.3679,0.1839,0.0613,0.0153


In [502]:
#model poisoon for different K for different states for deaths

K = [0,1,2,3,4]
NCD = []


# append calculated k to each corresponding list
for county in counties:
    countyK = []
    for num in K:
        PMF = stats.poisson.pmf(k=num, mu=NCNormDeaths_week[county +" Deaths(Norm)"]).round(4)
        countyK.append(PMF)
    NCD.append(countyK)
          
# NC

# #create a dataframe from lists

NCDeaths = pd.DataFrame(NC, columns = ['K=0', 'K=1', 'K=2','K=3','K=4']) 
NCDeaths["County Name"] = counties
NCDeaths.set_index("County Name", inplace = True)
NCDeaths.head()

,K=0,K=1,K=2,K=3,K=4
County Name,,,,,
Alamance,0.0498,0.1494,0.2240,0.2240,0.1680
Alexander,0.1353,0.2707,0.2707,0.1804,0.0902
Alleghany,0.0498,0.1494,0.2240,0.2240,0.1680
Anson,0.0498,0.1494,0.2240,0.2240,0.1680
Ashe,0.3679,0.3679,0.1839,0.0613,0.0153


## Perform corelation between Enrichment data valiables and COVID-19 cases to observe any patterns

In [513]:
# Read csv file 
demographic_df = pd.read_csv("../../../data/Demographic.csv")

# Remove columns which shows margin of errors, percent estimate and percent margin of error 
demographic_df = demographic_df[demographic_df.columns.drop(list(demographic_df.filter(regex='Margin')))]
demographic_df = demographic_df[demographic_df.columns.drop(list(demographic_df.filter(regex='Percent')))]
demographic_df

,GEO_ID,NAME,DP05_0031PM,DP05_0032E,DP05_0032M,DP05_0032PE,DP05_0032PM,DP05_0033E,DP05_0033M,DP05_0033PE,...,DP05_0029M,DP05_0029PE,DP05_0029PM,DP05_0030E,DP05_0030M,DP05_0030PE,DP05_0030PM,DP05_0031E,DP05_0031M,DP05_0031PE
0,id,Geographic Area Name,Percent Margin of Error!!SEX AND AGE!!Total po...,Estimate!!SEX AND AGE!!Total population!!65 ye...,Margin of Error!!SEX AND AGE!!Total population...,Percent Estimate!!SEX AND AGE!!Total populatio...,Percent Margin of Error!!SEX AND AGE!!Total po...,Estimate!!RACE!!Total population,Margin of Error!!RACE!!Total population,Percent Estimate!!RACE!!Total population,...,Margin of Error!!SEX AND AGE!!Total population...,Percent Estimate!!SEX AND AGE!!Total populatio...,Percent Margin of Error!!SEX AND AGE!!Total po...,Estimate!!SEX AND AGE!!Total population!!65 ye...,Margin of Error!!SEX AND AGE!!Total population...,Percent Estimate!!SEX AND AGE!!Total populatio...,Percent Margin of Error!!SEX AND AGE!!Total po...,Estimate!!SEX AND AGE!!Total population!!65 ye...,Margin of Error!!SEX AND AGE!!Total population...,Percent Estimate!!SEX AND AGE!!Total populatio...
1,0500000US01001,"Autauga County, Alabama",0.4,76.5,1.2,(X),(X),55200,*****,55200,...,75,8050,(X),3488,30,43.3,0.4,4562,65,56.7
2,0500000US01003,"Baldwin County, Alabama",0.1,86.6,0.3,(X),(X),208107,*****,208107,...,69,40665,(X),18870,40,46.4,0.1,21795,56,53.6
3,0500000US01005,"Barbour County, Alabama",0.2,76.2,0.6,(X),(X),25782,*****,25782,...,29,4634,(X),2004,16,43.2,0.2,2630,19,56.8
4,0500000US01007,"Bibb County, Alabama",1.3,80.5,4.3,(X),(X),22527,*****,22527,...,161,3661,(X),1633,74,44.6,1.3,2028,114,55.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3216,0500000US72145,"Vega Baja Municipio, Puerto Rico",*****,80.5,*****,(X),(X),53371,*****,53371,...,*****,9963,(X),4442,*****,44.6,*****,5521,*****,55.4
3217,0500000US72147,"Vieques Municipio, Puerto Rico",*****,87.1,*****,(X),(X),8771,*****,8771,...,*****,1826,(X),850,*****,46.5,*****,976,*****,53.5
3218,0500000US72149,"Villalba Municipio, Puerto Rico",*****,81.7,*****,(X),(X),22993,*****,22993,...,*****,3647,(X),1640,*****,45,*****,2007,*****,55
3219,0500000US72151,"Yabucoa Municipio, Puerto Rico",*****,85.2,*****,(X),(X),34149,*****,34149,...,*****,6340,(X),2916,*****,46,*****,3424,*****,54


In [511]:
#select county FIPS from is columns and append them in the list and create new column with the list 

county_FIPS_list = []
for i in range(len(demographic_df["id"])):
    county_FIPS_list.append(demographic_df.loc[i,'id'][9:])
    
demographic_df["countyFIPS"] = county_FIPS_list

# convert the type of countyFIPS from string to int
demographic_df['countyFIPS'] = demographic_df['countyFIPS'].astype(int)

# Split geographical area name into county and state name
demographic_df[['County Name','State']] = demographic_df["Geographic Area Name"].str.split(",",expand=True)

demographic_df.drop(columns = ['id', 'Geographic Area Name'], inplace = True)

KeyError: 'id'